# 如何将参数从一个步骤传递到下一个步骤

:::info 先决条件

本指南假定您熟悉以下概念：
- [LangChain Expression Language (LCEL)](/docs/concepts/lcel)
- [链接 runnables](/docs/how_to/sequence/)
- [并行调用 runnables](/docs/how_to/parallel/)
- [自定义函数](/docs/how_to/functions/)

:::

当组合具有多个步骤的链时，有时您希望将先前步骤的数据不加修改地传递，以用作后续步骤的输入。 [`RunnablePassthrough`](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html) 类允许您这样做，并且通常与 [RunnableParallel](/docs/how_to/parallel/) 结合使用，将数据传递到您构建的链中的后续步骤。

请参阅以下示例：

In [ ]:
%pip install -qU langchain langchain-openai

import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

runnable = RunnableParallel(
    passed=RunnablePassthrough(),
    modified=lambda x: x["num"] + 1,
)

runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'modified': 2}

如上所示，`passed` 键调用了 `RunnablePassthrough()`，因此它仅仅传递了 `{'num': 1}`。

我们在映射中设置了第二个键 `modified`。它使用了 lambda 函数对单个值进行设置，为 num 加 1，这导致 `modified` 键的值为 `2`。

## 检索示例

在下面的示例中，我们看到一个更贴近现实的用例，其中我们将 `RunnablePassthrough` 与 `RunnableParallel` 在一个链（chain）中结合使用，以正确格式化输入到提示（prompt）中的内容：

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

预期的系统提示输入是一个包含 "context" 和 "question" 键的映射。用户输入只有问题。因此，我们需要使用检索器获取上下文，并将用户输入作为 "question" 键的值传递。`RunnablePassthrough` 允许我们将用户的问题传递给提示和模型。

## 后续步骤

现在您已经学会了如何通过链传递数据，以帮助格式化在链中流动的数据。

要了解更多信息，请参阅本节中关于可运行对象的其他操作指南。